In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
charity_df = pd.read_csv("charity_data.csv")
charity_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
charity_df = charity_df.drop(["EIN", "NAME"], axis=1)

In [3]:
charity_cat = charity_df.dtypes[charity_df.dtypes == "object"].index.tolist()
charity_df[charity_cat].nunique()

APPLICATION_TYPE          17
AFFILIATION                6
CLASSIFICATION            71
USE_CASE                   5
ORGANIZATION               4
INCOME_AMT                 9
SPECIAL_CONSIDERATIONS     2
dtype: int64

In [4]:
application_count = charity_df.APPLICATION_TYPE.value_counts()
application_count

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
replace_application = list(application_count[application_count< 500].index)

# Replace in dataframe
for app in replace_application:
    charity_df.APPLICATION_TYPE = charity_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
charity_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
classification_count = charity_df.CLASSIFICATION.value_counts()
classification_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4200        1
C5200        1
C1248        1
C3700        1
C6100        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
replace_classification = list(classification_count[classification_count< 500].index)

# Replace in dataframe
for app in replace_classification:
    charity_df.CLASSIFICATION = charity_df.CLASSIFICATION.replace(app,"Other")
    
# Check to make sure binning was successful
charity_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [8]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(charity_df[charity_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(charity_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [9]:
# Merge one-hot encoded features and drop the originals
charity_df = charity_df.merge(encode_df, left_index=True, right_index=True)
charity_df = charity_df.drop(charity_cat, 1)
charity_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [10]:
# Split our preprocessed data into our features and target arrays
y = charity_df.IS_SUCCESSFUL.values
X = charity_df.drop(columns='IS_SUCCESSFUL').values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78, stratify=y)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Two_Hidden_Layer_Attempt

In [12]:
# Define the model - deep neural net
number_input_features = len(X_train[0])

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=24, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=12, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 24)                1080      
_________________________________________________________________
dense_1 (Dense)              (None, 12)                300       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 13        
Total params: 1,393
Trainable params: 1,393
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [14]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 1s 721us/step - loss: 0.5901 - accuracy: 0.7063
Epoch 2/50
804/804 [==============================] - 1s 735us/step - loss: 0.5582 - accuracy: 0.7259
Epoch 3/50
804/804 [==============================] - 1s 899us/step - loss: 0.5546 - accuracy: 0.7283
Epoch 4/50
804/804 [==============================] - 1s 937us/step - loss: 0.5528 - accuracy: 0.72880s - loss: 0.5
Epoch 5/50
804/804 [==============================] - 1s 914us/step - loss: 0.5509 - accuracy: 0.72940s - los
Epoch 6/50
804/804 [==============================] - 1s 742us/step - loss: 0.5501 - accuracy: 0.7288
Epoch 7/50
804/804 [==============================] - 1s 943us/step - loss: 0.5491 - accuracy: 0.73150s - loss: 0.5495 - accu
Epoch 8/50
804/804 [==============================] - 1s 764us/step - loss: 0.5478 - accuracy: 0.7322
Epoch 9/50
804/804 [==============================] - 1s 754us/step - loss: 0.5472 - accuracy: 0.7324
Epoch 10/50
804/804 [===============

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 15361.8223 - accuracy: 0.5324
Loss: 15361.822265625, Accuracy: 0.5323615074157715


# Three_Hidden_Layer_Attempt

In [12]:
# Define the model - deep neural net
number_input_features = len(X_train[0])

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=50, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=10, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 50)                2250      
_________________________________________________________________
dense_1 (Dense)              (None, 30)                1530      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                310       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
Total params: 4,101
Trainable params: 4,101
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [14]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5712 - accuracy: 0.7190
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5537 - accuracy: 0.7290: 0s - loss: 0.5496 
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5517 - accuracy: 0.7302: 0s - loss: 0.5530 - accuracy: 
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5503 - accuracy: 0.7300
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5491 - accuracy: 0.7318
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5483 - accuracy: 0.7308
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5479 - accuracy: 0.7324
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5470 - accuracy: 0.7321
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5471 - accuracy: 0.7319
Epoch 10/100
804/804 [=================

804/804 [==============================] - 1s 2ms/step - loss: 0.5354 - accuracy: 0.7383
Epoch 78/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5360 - accuracy: 0.7383
Epoch 79/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5358 - accuracy: 0.7386
Epoch 80/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5356 - accuracy: 0.7378
Epoch 81/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5354 - accuracy: 0.7386
Epoch 82/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5354 - accuracy: 0.7390
Epoch 83/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5355 - accuracy: 0.7390
Epoch 84/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5357 - accuracy: 0.7384
Epoch 85/100
804/804 [==============================] - ETA: 0s - loss: 0.5354 - accuracy: 0.73 - 2s 2ms/step - loss: 0.5352 - accuracy: 0.7388
Epoch 86/100
804/804 [==============================]

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 7196.0825 - accuracy: 0.5324
Loss: 7196.08251953125, Accuracy: 0.5323615074157715


# One_Hidden_Layer

In [16]:
# Define the model - deep neural net
number_input_features = len(X_train[0])

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=30, input_dim=number_input_features, activation="relu")
)

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 30)                1350      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 31        
Total params: 1,381
Trainable params: 1,381
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [18]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5843 - accuracy: 0.7109
Epoch 2/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5595 - accuracy: 0.7276
Epoch 3/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5563 - accuracy: 0.7279
Epoch 4/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5546 - accuracy: 0.7286
Epoch 5/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5529 - accuracy: 0.7284: 0s - loss: 0.5524 - ac
Epoch 6/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5514 - accuracy: 0.7291
Epoch 7/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5510 - accuracy: 0.7293
Epoch 8/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5503 - accuracy: 0.7293
Epoch 9/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5497 - accuracy: 0.7301
Epoch 10/50
804/804 [==============================] - 1s 2ms/step - loss: 0

In [19]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 89381.7500 - accuracy: 0.5324
Loss: 89381.75, Accuracy: 0.5323615074157715


# Checkpoint

In [49]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [17]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=1000)

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
804/804 [==============================] - 1s 716us/step - loss: 0.5404 - accuracy: 0.7346
Epoch 2/100
128/804 [===>..........................] - ETA: 0s - loss: 0.5314 - accuracy: 0.7451
Epoch 00002: saving model to checkpoints/weights.02.hdf5
804/804 [==============================] - 1s 794us/step - loss: 0.5394 - accuracy: 0.7369
Epoch 3/100
364/804 [============>.................] - ETA: 0s - loss: 0.5391 - accuracy: 0.7351
Epoch 00003: saving model to checkpoints/weights.03.hdf5
804/804 [==============================] - 1s 825us/step - loss: 0.5400 - accuracy: 0.7359
Epoch 4/100
572/804 [====================>.........] - ETA: 0s - loss: 0.5422 - accuracy: 0.7365
Epoch 00004: saving model to checkpoints/weights.04.hdf5
804/804 [==============================] - 1s 727us/step - loss: 0.5396 - accuracy: 0.7371
Epoch 5/100
722/804 [=========================>....] - ETA: 0s - loss: 0.5396 - accuracy: 0.7375
Epoch 00005: saving model to checkpoints/weights.05.hdf5
804/804 

804/804 [==============================] - 1s 742us/step - loss: 0.5377 - accuracy: 0.7369
Epoch 39/100
426/804 [==============>...............] - ETA: 0s - loss: 0.5316 - accuracy: 0.7412
Epoch 00039: saving model to checkpoints/weights.39.hdf5
804/804 [==============================] - 1s 744us/step - loss: 0.5379 - accuracy: 0.7375
Epoch 40/100
634/804 [======================>.......] - ETA: 0s - loss: 0.5389 - accuracy: 0.7372
Epoch 00040: saving model to checkpoints/weights.40.hdf5
804/804 [==============================] - 1s 765us/step - loss: 0.5375 - accuracy: 0.7379
Epoch 41/100
804/804 [==============================] - 1s 730us/step - loss: 0.5377 - accuracy: 0.7376
Epoch 42/100
  1/804 [..............................] - ETA: 0s - loss: 0.6015 - accuracy: 0.7188
Epoch 00042: saving model to checkpoints/weights.42.hdf5
804/804 [==============================] - 1s 738us/step - loss: 0.5375 - accuracy: 0.7362
Epoch 43/100
214/804 [======>.......................] - ETA: 0s - l

804/804 [==============================] - 1s 760us/step - loss: 0.5364 - accuracy: 0.7382
Epoch 76/100
697/804 [=========================>....] - ETA: 0s - loss: 0.5366 - accuracy: 0.7378
Epoch 00076: saving model to checkpoints/weights.76.hdf5
804/804 [==============================] - 1s 747us/step - loss: 0.5367 - accuracy: 0.7380
Epoch 77/100
804/804 [==============================] - 1s 763us/step - loss: 0.5364 - accuracy: 0.7375
Epoch 78/100
 44/804 [>.............................] - ETA: 0s - loss: 0.5502 - accuracy: 0.7180
Epoch 00078: saving model to checkpoints/weights.78.hdf5
804/804 [==============================] - 1s 775us/step - loss: 0.5362 - accuracy: 0.7378
Epoch 79/100
216/804 [=======>......................] - ETA: 0s - loss: 0.5341 - accuracy: 0.7409
Epoch 00079: saving model to checkpoints/weights.79.hdf5
804/804 [==============================] - 1s 757us/step - loss: 0.5363 - accuracy: 0.7392
Epoch 80/100
430/804 [===============>..............] - ETA: 0s - l

In [43]:
X_train.shape

(25724, 44)

In [44]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 24
hidden_nodes_layer2 = 12

nn_new = tf.keras.models.Sequential()

# First hidden layer
nn_new.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn_new.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn_new.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))


# Compile the model
nn_new.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Restore the model weights
nn_new.load_weights("checkpoints/weights.65.hdf5")

# Evaluate the model using the test data
model_loss, model_accuracy = nn_new.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5523 - accuracy: 0.7332
Loss: 0.5522703528404236, Accuracy: 0.7331778407096863


In [19]:
# Export our model to HDF5 file
nn_new.save("trained_attrition.h5")

In [20]:
# Import the model to a new object
nn_imported = tf.keras.models.load_model('trained_attrition.h5')

In [21]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_new.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5534 - accuracy: 0.7308
Loss: 0.5534027218818665, Accuracy: 0.7308454513549805
